In [3]:
import os

In [12]:
os.environ["OPENAI_API_KEY"] = "your key"
os.environ["VIDEO_DB_API_KEY"] = "your key"

In [2]:
%pip install llama-index
%pip install videodb
%pip install llama-index-retrievers-videodb

Defaulting to user installation because normal site-packages is not writeable
  Using cached Deprecated-1.2.14-py2.py3-none-any.whl.metadata (5.4 kB)
     ---------------------------------------- 0.0/67.3 kB ? eta -:--:--
     ---------------------------------------- 67.3/67.3 kB ? eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached websocket_client-1.7.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.met

In [13]:
from videodb import connect

# connect to VideoDB
conn = connect()

# upload videos to default collection in VideoDB
print("uploading first video: Benoît Mandelbrot")
video1 = conn.upload(url="https://www.youtube.com/watch?v=ay8OMOsf6AQ")


uploading first video: Benoît Mandelbrot


In [14]:
print("Indexing the video...")
video1.index_spoken_words()

Indexing the video...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:17<00:00,  5.61it/s]


In [11]:
from llama_index.retrievers.videodb import VideoDBRetriever
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine

In [15]:
# VideoDBRetriever by default uses the default collection in the VideoDB
retriever = VideoDBRetriever()

# use your llama_index response_synthesizer on search results.
response_synthesizer = get_response_synthesizer()

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [16]:
# query across all uploaded videos to get the text answer.
response = query_engine.query("What is the Mandelbrot Set?")
print(response)

The Mandelbrot Set is a shape that arises from an exercise in pure mathematics. It is a set of complex numbers that, when iterated through a particular mathematical formula, either remain bounded or diverge to infinity. The intricate and visually striking patterns of the Mandelbrot Set emerge from the repeated application of simple rules.


In [17]:
from videodb import connect, play_stream
from videodb.timeline import Timeline
from videodb.asset import VideoAsset

# create video stream of search results

In [20]:
conn = connect()
timeline = Timeline(conn)

relevant_nodes = retriever.retrieve("What is the Mandelbrot set?")

for node_obj in relevant_nodes:
    node = node_obj.node
    # create a video asset for each node
    node_asset = VideoAsset(
        asset_id=node.metadata["video_id"],
        start=node.metadata["start"],
        end=node.metadata["end"],
    )
    # add the asset to timeline
    timeline.add_inline(node_asset)

# generate stream for the compiled timeline
stream_url = timeline.generate_stream()
play_stream(stream_url)

'https://console.videodb.io/player?url=https://stream.videodb.io/v3/published/manifests/2ba7fd24-acab-474f-9733-5ebfd2b03601.m3u8'

# Clean UP

In [21]:
video1.delete()